In [3]:
# prompt: gere um conjunto de funções para extrair e processar informações de um determinado site. O endereço base é base_url = "http://www.referencialcurriculardoparana.pr.gov.br/modules/conteudo/conteudo.php?conteudo=" onde conteúdo será seguido por um numero, exemplo 33. Crie uma função para acessar a página e coletar todo o seu conteúdo html. Dê um intervalo de 2 segundos depois de ler o conteúdo para garantir que não tome block de toomanyrequests. Estaremos interessados posteriormente em extrair o h1 e as tbody dele. Para cada tbody encontrado extraia o conteúdo das linhas e colunas como uma lista de listas.  Adicionar 2 novas chaves content_id com o id da página e h1 com o valor extraído. De forma que o resultado de cada chamada seja um dicionário do tipo {content_id: {h1: x, bodies: [[]]}}. Crie uma segunda função para iterar sob um range de codigos, gerenciando as chamadas e juntando os resultados recuperados.

import requests
from bs4 import BeautifulSoup
import time

def extract_page_data(content_id):
  """Extracts data from a single page.

  Args:
    content_id: The ID of the content to extract.

  Returns:
    A dictionary containing the extracted data, or None if an error occurs.
  """
  base_url = "http://www.referencialcurriculardoparana.pr.gov.br/modules/conteudo/conteudo.php?conteudo="
  url = f"{base_url}{content_id}"

  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
  except requests.exceptions.RequestException as e:
    print(f"Error fetching page {content_id}: {e}")
    return None

  soup = BeautifulSoup(response.content, 'html.parser')

  h1_tag = soup.find('h1')
  h1_content = h1_tag.get_text(strip=True) if h1_tag else None

  bodies_data = []
  for tbody in soup.find_all('tbody'):
    tbody_data = []
    for row in tbody.find_all('tr'):
      row_data = [col.get_text(strip=True) for col in row.find_all(['td', 'th'])]
      tbody_data.append(row_data)
    bodies_data.append(tbody_data)

  time.sleep(2) # Wait for 2 seconds

  return {
      "content_id": content_id,
      "h1": h1_content,
      "bodies": bodies_data
  }

def extract_data_from_range(start_id, end_id):
  """Extracts data from a range of content IDs.

  Args:
    start_id: The starting content ID (inclusive).
    end_id: The ending content ID (inclusive).

  Returns:
    A dictionary where keys are content IDs and values are the extracted data
    dictionaries.
  """
  all_data = {}
  for content_id in range(start_id, end_id + 1):
    page_data = extract_page_data(content_id)
    if page_data:
      all_data[content_id] = page_data
  return all_data



In [4]:
raw_data = extract_data_from_range(30, 111)
raw_data

{30: {'content_id': 30,
  'h1': 'Arte - 1º ano',
  'bodies': [[['UNIDADE TEMÁTICA',
     'OBJETOS DE CONHECIMENTO',
     'OBJETIVOS DE APRENDIZAGEM'],
    ['Artes Visuais',
     'Contextos e práticas',
     '(EF15AR01) Identificar e apreciar formas distintas das artes visuais tradicionais e contemporâneas, cultivando a percepção, o imaginário, a capacidade de simbolizar e o repertório imagético.Conhecer e perceber os diferentes gêneros da arte como: retrato e autorretrato, paisagem, natureza morta, cenas da mitologia, cenas religiosas e cenas históricas e dos diferentes contextos históricos/artísticos comparando-os a partir das diferenças formais.'],
    ['Artes Visuais',
     'Elementos da linguagem',
     '(EF15AR02) Explorar e reconhecer elementos constitutivos das artes visuais (ponto, linha, forma, cor, espaço, movimento etc.).Conhecer, reconhecer e explorar os elementos da linguagem visual (ponto, linha, forma, cor, volume, superfície, presentes na natureza, nas obras de arte e i

In [47]:
import re
import unicodedata


def get_years(code):
  try:
    start, end = int(code[2]), int(code[3])
  except:
    start, end = None, None
  return start, end


def extract_codes_and_values(text):
    """
    Extracts (codigo, valor) pairs where codigo matches the WWDDWWDD format.
    Args:
        text (str): The input string.
    Returns:
        List[Tuple[str, str]]: A list of (codigo, valor) tuples.
    """
    # Regex que identifica separadores válidos no formato (WWDDWWDD)
    pattern = r'\(([A-Z]{2}\d{2}[A-Z]{2}\d{2})\)'

    # Encontra todos os índices onde há separadores válidos
    matches = list(re.finditer(pattern, text))

    results = []

    for i, match in enumerate(matches):
        codigo = match.group(1)
        start = match.end()
        end = matches[i+1].start() if i + 1 < len(matches) else len(text)
        valor = text[start:end].strip()
        results.append((codigo, valor))

    return results


def remove_accents(input_str):
    """Removes accented characters from a string.

    Args:
        input_str: The input string with potential accented characters.

    Returns:
        The string with accented characters replaced by their non-accented equivalents.
    """
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])



entries = list()
codes = list()
for content_id, v in raw_data.items():
    h1 = v['h1']
    subject = h1.split(' - ')[0].strip()
    subject_normalized = remove_accents(subject).lower().replace(' ', '_')
    school_years = h1.split(' - ')[1].split(' ao ')
    school_years = [s.split('º')[0] for s in school_years]

    bodies = v['bodies']
    for b in bodies:
      for r in b:
        if len(r) == 3 and r[0] != 'UNIDADE TEMÁTICA' and r[0] != 'PRÁTICAS DE LINGUAGEM' and r[2] != 'OBJETIVOS DE APRENDIZAGEM':
          unidade_tematica = r[0]
          objeto_de_conhecimento = r[1]
          objetivo_de_aprendizagem = r[2]
        elif len(r) == 4 and r[0] != 'UNIDADE TEMÁTICA' and r[0] != 'PRÁTICAS DE LINGUAGEM' and r[0] != 'CAMPOS DE ATUAÇÃO':
          unidade_tematica = r[1] + ' - ' + r[0]
          objeto_de_conhecimento = r[2]
          objetivo_de_aprendizagem = r[3]
        else:
          continue

        extracted_pairs = extract_codes_and_values(objetivo_de_aprendizagem)
        for codigo_objetivo_de_aprendizagem, objetivo_de_aprendizagem in extracted_pairs:
          if len(codigo_objetivo_de_aprendizagem) == 8 and codigo_objetivo_de_aprendizagem.startswith('EF') and len(objetivo_de_aprendizagem) > 10:
            school_year_start, school_year_end = get_years(codigo_objetivo_de_aprendizagem)
            if school_year_start == 0:
              school_year_start = school_year_end
            if codigo_objetivo_de_aprendizagem not in codes:
              codes.append(codigo_objetivo_de_aprendizagem)
              entries.append([content_id, subject, subject_normalized, school_year_start, school_year_end,
                              unidade_tematica, objeto_de_conhecimento,
                              codigo_objetivo_de_aprendizagem, objetivo_de_aprendizagem.strip().lower()])
            else:
              print('\ncódigo repetido', unidade_tematica, codigo_objetivo_de_aprendizagem, objetivo_de_aprendizagem)
          else:
            print('\ndescartado o código', unidade_tematica, codigo_objetivo_de_aprendizagem, objetivo_de_aprendizagem)


código repetido Artes Visuais EF15AR01 Identificar e apreciar formas distintas das artes visuais tradicionais e contemporâneas, cultivando a percepção, o imaginário, a capacidade de simbolizar e o repertório imagético.Conhecer e perceber os diferentes gêneros da arte como: retrato e autorretrato, paisagem, natureza morta, cenas da mitologia, cenas religiosas e cenas históricas e dos diferentes contextos históricos/artísticos comparando-os a partir das diferenças formais.Conhecer e apreciar a produção artística de artistas locais ou regionais para compreender a realidade histórica e cultural regional.

código repetido Artes Visuais EF15AR02 Explorar e reconhecer elementos constitutivos das artes visuais (ponto, linha, forma, cor, espaço, movimento etc.).Conhecer, reconhecer e explorar os elementos da linguagem visual (ponto, linha, forma, cor, volume, Superfície, presentes na natureza, nas obras de arte e imagens do cotidiano, para elaborar composições artísticas tanto no bidimensional

In [49]:
# prompt: imprima entries como um pd dataframe. agrupe por codigo_objetivo_de_aprendizagem, conte, ordene pela contagem decrescente. filtre apenas quando codigo EF15AR01

import pandas as pd

df = pd.DataFrame(entries, columns=['content_id', 'subject', 'subject_normalized', 'school_year_start', 'school_year_end', 'unidade_tematica', 'objeto_de_conhecimento', 'codigo_objetivo_de_aprendizagem', 'objetivo_de_aprendizagem'])

filtered_df = df[df['codigo_objetivo_de_aprendizagem'].str.startswith('EF15AR01')]

grouped_df = filtered_df.groupby(['codigo_objetivo_de_aprendizagem']).size().reset_index(name='count')

sorted_df = grouped_df.sort_values(by='count', ascending=False)

# sorted_df
# filtered_df
df

,content_id,subject,subject_normalized,school_year_start,school_year_end,unidade_tematica,objeto_de_conhecimento,codigo_objetivo_de_aprendizagem,objetivo_de_aprendizagem
0,30,Arte,arte,1,5,Artes Visuais,Contextos e práticas,EF15AR01,identificar e apreciar formas distintas das ar...
1,30,Arte,arte,1,5,Artes Visuais,Elementos da linguagem,EF15AR02,explorar e reconhecer elementos constitutivos ...
2,30,Arte,arte,1,5,Artes Visuais,Matrizes estéticas e culturais,EF15AR03,reconhecer e analisar a influência de distinta...
3,30,Arte,arte,1,5,Artes Visuais,Materialidades,EF15AR04,experimentar diferentes formas de expressão ar...
4,30,Arte,arte,1,5,Artes Visuais,Processos de criação,EF15AR05,experimentar a criação em artes visuais de mod...
...,...,...,...,...,...,...,...,...,...
1288,111,Matemática,matematica,9,9,Grandezas e Medidas,Medidas de volume,EF09MA19,"resolver e elaborar problemas, de diferentes c..."
1289,111,Matemática,matematica,9,9,Tratamento da Informação,Noções de probabilidade,EF09MA20,"reconhecer e compreender, em experimentos alea..."
1290,111,Matemática,matematica,9,9,Tratamento da Informação,GráficoInformação,EF09MA21,"ler, interpretar, analisar e identificar, em g..."
1291,111,Matemática,matematica,9,9,Tratamento da Informação,GráficoInformaçãoEstatística,EF09MA22,"escolher e construir gráfico (colunas, linhas,..."


In [50]:
def sanitize_string(s):
    """Sanitizes a string for SQL insertion by escaping dangerous characters."""
    if isinstance(s, str):
        return s.replace("'", "''").replace('"', ' ')
    return s

# Gera a instrução SQL completa com os valores interpolados
sql_statements = []

for entry in entries:
    sanitized_entry = [sanitize_string(item) for item in entry]
    # Aspas em valores string, sem aspas em inteiros
    formatted_values = [
        f"'{item}'" if isinstance(item, str) else str(item)
        for item in sanitized_entry
    ]
    sql_line = f'''INSERT INTO bncc_guidelines (content_id, subject, subject_normalized, age_start, age_end,thematic_unit, knowledge_object, knowledge_code, knowledge) VALUES ({", ".join(formatted_values)});'''
    sql_statements.append(sql_line.strip())

In [52]:
import psycopg2
import os


# Fetch variables
USER = ''
PASSWORD = ''
HOST = ''
PORT = 6543
DBNAME = 'postgres'

try:
    connection = psycopg2.connect(
        user=USER,
        password=PASSWORD,
        host=HOST,
        port=PORT,
        dbname=DBNAME
    )
    print("Connection successful!")

    # Create a cursor to execute SQL queries
    cursor = connection.cursor()

    for sql in sql_statements:
      print(sql)
      cursor.execute(sql)
      # result = cursor.fetchone()
      # print("Current Time:", result)

    connection.commit()

    # Close the cursor and connection
    cursor.close()
    connection.close()
    print("Connection closed.")

except Exception as e:
    print(f"Failed to connect: {e}")

Connection successful!
INSERT INTO bncc_guidelines (content_id, subject, subject_normalized, age_start, age_end,thematic_unit, knowledge_object, knowledge_code, knowledge) VALUES (30, 'Arte', 'arte', 1, 5, 'Artes Visuais', 'Contextos e práticas', 'EF15AR01', 'identificar e apreciar formas distintas das artes visuais tradicionais e contemporâneas, cultivando a percepção, o imaginário, a capacidade de simbolizar e o repertório imagético.conhecer e perceber os diferentes gêneros da arte como: retrato e autorretrato, paisagem, natureza morta, cenas da mitologia, cenas religiosas e cenas históricas e dos diferentes contextos históricos/artísticos comparando-os a partir das diferenças formais.');
INSERT INTO bncc_guidelines (content_id, subject, subject_normalized, age_start, age_end,thematic_unit, knowledge_object, knowledge_code, knowledge) VALUES (30, 'Arte', 'arte', 1, 5, 'Artes Visuais', 'Elementos da linguagem', 'EF15AR02', 'explorar e reconhecer elementos constitutivos das artes visua